In [6]:
!pip install langchain
!pip install tiktoken
!pip install -U langchain langchain-community
!pip install -U langchain langchain-core langchain-openai
!pip install chromadb
!pip install -U langchain sentence-transformers chromadb
!pip install -U sentence-transformers langchain
!pip install pymupdf
!pip install faiss-cpu

KeyboardInterrupt: 

### RAG - Retriever
  - Vector Store Retriver
  - Multi Query Retriever
  - Contextual compression

In [4]:
#Vector Store Retriver

from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings


from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
os.environ['OPENAI_API_KEY'] = ''

In [9]:

#사전 준비 - 문서 로드 및 분할


loader = PyMuPDFLoader("/content/drive/MyDrive/Colab Notebooks/PDF_예제.pdf")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200,
    encoding_name="cl100k_base"
)

documents = text_splitter.split_documents(data)
len(documents)


38

In [12]:
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sbert-nli",
    model_kwargs={'device' : 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
vectorstore = FAISS.from_documents(
    documents,
    embedding = embeddings_model,
    distance_strategy = DistanceStrategy.COSINE
)


In [16]:

# 단일 문서 검색

query = "기업이 자율주행을 어떻게 연구하는지 알려줘"

retriever = vectorstore.as_retriever(search_kwargs={'k':1})


docs = retriever.get_relevant_documents(query)
print(len(docs))
docs[0]

1


/tmp/ipython-input-16-1338684130.py:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


Document(id='a5b54b4c-62a4-49cf-801d-cc31327f88fc', metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hancom PDF 1.3.0.538', 'creationdate': '2021-11-11T21:30:57+09:00', 'source': '/content/drive/MyDrive/Colab Notebooks/PDF_예제.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/PDF_예제.pdf', 'total_pages': 22, 'format': 'PDF 1.4', 'title': '', 'author': 'kmd kdy', 'subject': '', 'keywords': '', 'moddate': '2021-11-11T21:30:57+09:00', 'trapped': '', 'modDate': "D:20211111213057+09'00'", 'creationDate': "D:20211111213057+09'00'", 'page': 14}, page_content='자율주행 자동차 기술·시장 동향\n  CIS이슈리포트2021-4호 | 15 |\n[자율주행 자동차 해외 업체 현황(완성차 업계)]\n업체\n현황\nŸ Ford는 약 8,400억 원을 투입하여 자율주행 자동차 제조혁신센터를 설립하는 등 전기차와 \n상용화를 위해 약 5.4조 원을 투입함\nŸ 자율주행 스타트업 기업인 Argo AI를 10억 달러에 인수하였으며, Velodyne에 7,500만 달러를 \n투자하여 협업 중임 \nŸ 2021년 미국 3개 주에서 본격적인 시험에 돌입하였음 \nŸ Volkswagen은 소프트웨어 자체개발 비중을 현재 10% 미만에서 2025년 최소 60% 수준으로 \n확보할 계획이며, 2025년부터 완전 자율주행 자동차 상용화 목표를 제시함 \nŸ 2017년 자회사 Audi의 신형 A8에 레벨3 자율주행기술을 탑재하였으며

In [20]:
#MMR 검색 - 다양성을 고려한 방식 (lambda_mult = 0.5)

retriever = vectorstore.as_retriever(
    search_type = 'mmr',
    search_kwargs = {'k':5, 'fetch_k':50}
)

docs = retriever.get_relevant_documents(query)
print(docs[0])
print("\n\n\n")
print(docs[-1])



page_content='자율주행 자동차 기술·시장 동향
  CIS이슈리포트2021-4호 | 15 |
[자율주행 자동차 해외 업체 현황(완성차 업계)]
업체
현황
Ÿ Ford는 약 8,400억 원을 투입하여 자율주행 자동차 제조혁신센터를 설립하는 등 전기차와 
상용화를 위해 약 5.4조 원을 투입함
Ÿ 자율주행 스타트업 기업인 Argo AI를 10억 달러에 인수하였으며, Velodyne에 7,500만 달러를 
투자하여 협업 중임 
Ÿ 2021년 미국 3개 주에서 본격적인 시험에 돌입하였음 
Ÿ Volkswagen은 소프트웨어 자체개발 비중을 현재 10% 미만에서 2025년 최소 60% 수준으로 
확보할 계획이며, 2025년부터 완전 자율주행 자동차 상용화 목표를 제시함 
Ÿ 2017년 자회사 Audi의 신형 A8에 레벨3 자율주행기술을 탑재하였으며, 2021년 완전 자율주행
기술을 적용한 최고급 전기 세단을 출시를 계획하고 있음 
Ÿ Daimler Mercedes-Benz는 2020년 고속도로 등 특정 환경에서 자율주행되는 레벨3수준의 
DRIVE PILOT 시스템을 출시함. Bosch와 협업하고 있으며, 완성차 회사 BMW, Audi와 파
트너쉽을 맺음 
Ÿ 상용차의 경우, 고속도로 파일럿 시스템을 장착한 ‘벤츠 퓨처 트럭 2025’로 자율주행을 테스트 
중이며, 네덜란드에서 시범 운행을 시작함 
Ÿ Honda는 2016년 Waymo와 손잡고 자율주행 자동차 공동연구를 위한 검토에 돌입
하였으며, Waymo의 자율주행 전용 센서, 소프트웨어 등을 자사의 차에 탑재, 미국 내 일반
도로에서 완전 자율주행 실증실험에 들어감 
Ÿ GM은 완성차업체 중 가장 높은 자율주행기술력을 보유한 것으로 평가받고 있으며, 차량공유
업체인 Lyft와 자율주행기술을 보유한 Cruise Automation 스타트업 인수 등 공격적 행보를 
보이고 있음 
Ÿ 2017년 일부 캐딜락 모델에 자율주행시스템 ‘슈퍼 크루즈’를 옵션으로 장착하였음 
Ÿ 2018년부터 수천 대 규모의 자율주행

In [19]:
#MMR 검색 - 다양성을 고려한 방식 (lambda_mult가 작을수록 더 다양하게 추출)

retriever = vectorstore.as_retriever(
    search_type = 'mmr',
    search_kwargs = {'k':5, 'lambda_mult' : 0.15}
)

docs = retriever.get_relevant_documents(query)
print(docs[0])
print("\n\n\n")
print(docs[-1])

page_content='자율주행 자동차 기술·시장 동향
  CIS이슈리포트2021-4호 | 15 |
[자율주행 자동차 해외 업체 현황(완성차 업계)]
업체
현황
Ÿ Ford는 약 8,400억 원을 투입하여 자율주행 자동차 제조혁신센터를 설립하는 등 전기차와 
상용화를 위해 약 5.4조 원을 투입함
Ÿ 자율주행 스타트업 기업인 Argo AI를 10억 달러에 인수하였으며, Velodyne에 7,500만 달러를 
투자하여 협업 중임 
Ÿ 2021년 미국 3개 주에서 본격적인 시험에 돌입하였음 
Ÿ Volkswagen은 소프트웨어 자체개발 비중을 현재 10% 미만에서 2025년 최소 60% 수준으로 
확보할 계획이며, 2025년부터 완전 자율주행 자동차 상용화 목표를 제시함 
Ÿ 2017년 자회사 Audi의 신형 A8에 레벨3 자율주행기술을 탑재하였으며, 2021년 완전 자율주행
기술을 적용한 최고급 전기 세단을 출시를 계획하고 있음 
Ÿ Daimler Mercedes-Benz는 2020년 고속도로 등 특정 환경에서 자율주행되는 레벨3수준의 
DRIVE PILOT 시스템을 출시함. Bosch와 협업하고 있으며, 완성차 회사 BMW, Audi와 파
트너쉽을 맺음 
Ÿ 상용차의 경우, 고속도로 파일럿 시스템을 장착한 ‘벤츠 퓨처 트럭 2025’로 자율주행을 테스트 
중이며, 네덜란드에서 시범 운행을 시작함 
Ÿ Honda는 2016년 Waymo와 손잡고 자율주행 자동차 공동연구를 위한 검토에 돌입
하였으며, Waymo의 자율주행 전용 센서, 소프트웨어 등을 자사의 차에 탑재, 미국 내 일반
도로에서 완전 자율주행 실증실험에 들어감 
Ÿ GM은 완성차업체 중 가장 높은 자율주행기술력을 보유한 것으로 평가받고 있으며, 차량공유
업체인 Lyft와 자율주행기술을 보유한 Cruise Automation 스타트업 인수 등 공격적 행보를 
보이고 있음 
Ÿ 2017년 일부 캐딜락 모델에 자율주행시스템 ‘슈퍼 크루즈’를 옵션으로 장착하였음 
Ÿ 2018년부터 수천 대 규모의 자율주행

In [27]:
retriever = vectorstore.as_retriever(
    search_type = 'similarity_score_threshold',
    search_kwargs = {'score_threshold' : 0.3}

)

docs = retriever.get_relevant_documents(query)
print(len(docs))
print("\n\n\n")
print(docs[-1])

4




page_content='자율주행 자동차 기술·시장 동향
  CIS이슈리포트2021-4호 | 9 |
▶자율주행 시스템의 판단 및 제어기술은 인지기술을 통해 파악된 위치와 주변 환경 정보를 기반으로 
자율주행 차량의 현재 상황에 맞는 행동을 실행할 수 있도록 인공지능, 빅데이터, 시뮬레이션 및 
관련 SW 등이 적용되고 있음
￮
정밀 지도, 교통량, 도로 상황 등 환경 데이터를 종합하여 목적지까지의 최적화된 경로를 결정하고, 
주행 상황에 따라 속도, 주행미션(차선변경, 추월, 회피, 정지 등)을 결정하여 수행하게 됨
￮
실제 운전자가 조작하는 모든 장치에 대한 제어가 이루어져, 기존의 능동 차량제어기술인 구동제어, 
제동제어, 조향제어, 현가제어 등이 통합적으로 관리되기 때문에 판단 및 제어기술 부문에서는 고
신뢰성 이중 안전 센서 및 액추에이터 개발, 기능 안전 요구 사양기반 설계, 위험 분석 및 검증을 
통한 Fail Safety 제어기술 등 고도화가 이루어지고 있음
▶판단 및 제어관련 최신기술로는 인공지능 알고리즘, 빅데이터 분석, 경로 계획 및 생성 알고리즘 
등이 적용되고 있으며, 보행자/운전자/차량의 의도를 종합적으로 파악하여 주행 우선순위를 결정
하는 연구가 추진 중임
￮
기존의 규칙기반(Rule-based Approach) 제어기술은 산업 내 전문성을 갖춘 인력들이 오랜 시간
동안 정교하게 규칙을 모델링해야 되며, 실제 상황에서 지속적으로 테스트를 반복 및 검증해야 되는 
과정이 필요한 단점이 있음
￮
딥러닝 기술을 기반으로 한 판단 및 제어는 인간의 주행과정을 데이터화하여 인공지능이 주행 방법을 
학습하게 되므로 사람이 운전을 반복하는 과정을 통해 주행이 익숙해져 가는 것과 유사한 측면이 있음
- 학습 과정에서 일일이 방법을 정의할 필요 없이 상황별 달성 목표와 보상만 정의하면, 인공지능이 
다양한 상황에서의 대응 방법을 획득하고 터득하는 강화학습을 통해 최적의 대응 방법을 스스로 
깨우침
- 인지, 판단, 제어 등 단계를 구분하지 

In [25]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

retriever = vectorstore.as_retriever(
    search_type = 'mmr',
    search_kwargs = {'k': 5, 'lambda_mult':0.15}
)

docs = retriever.get_relevant_documents(query)

template = '''Answer the question based only on the following context:
{context}

Question: {question}'''

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature = 0,
    max_tokens=500,
)

def format_docs(docs):
  return '\n\n'.join([d.page_content for d in docs])

chain = prompt | llm | StrOutputParser()

response = chain.invoke({
    'context': format_docs(docs),
    'question': query
})

response

'기업들은 자율주행 기술을 연구하기 위해 다양한 방법과 기술을 활용하고 있습니다. 주요 연구 방법은 다음과 같습니다:\n\n1. **인공지능 및 빅데이터**: 자율주행 시스템의 판단 및 제어 기술은 인공지능과 빅데이터를 통해 차량의 위치와 주변 환경 정보를 분석하여 최적의 주행 경로를 결정합니다.\n\n2. **시뮬레이션 및 테스트**: 실제 도로에서의 시험 운전과 함께 시뮬레이션을 통해 다양한 주행 상황을 테스트하고 검증합니다. 예를 들어, Ford는 미국의 여러 주에서 자율주행 차량의 시험을 진행하고 있습니다.\n\n3. **협업 및 인수**: 기업들은 자율주행 스타트업을 인수하거나 협업하여 기술력을 강화하고 있습니다. 예를 들어, Ford는 Argo AI를 인수하고 Velodyne에 투자하여 협업하고 있습니다.\n\n4. **소프트웨어 개발**: Volkswagen은 소프트웨어 개발 비중을 늘려 완전 자율주행 자동차 상용화를 목표로 하고 있으며, 자율주행 기술을 자체 개발하기 위해 노력하고 있습니다.\n\n5. **고속도로 자율주행 기술**: Daimler Mercedes-Benz는 특정 환경에서 자율주행되는 시스템을 출시하고, 고속도로에서 자율주행을 테스트하는 차량을 개발하고 있습니다.\n\n6. **데이터 공유 및 협력**: 한국의 교통안전공단은 자율주행 데이터 공유센터를 운영하며, 여러 기관과 협력하여 데이터를 공유하고 있습니다.\n\n이러한 다양한 접근 방식을 통해 기업들은 자율주행 기술의 발전을 도모하고 있습니다.'